In [ ]:
from ct.mlexp1 import mlexp
from datetime import date

In [ ]:
import time
from IPython.display import Audio
sound_file = '/tmp/ding.wav'

In [ ]:
mlexp?

In [ ]:
t0 = time.time()
res=[] 
for sym in ["ETH"]:        
    for periods in [10,100,1000]:
        for qlags in [[100],[3000]]:
            resml=mlexp(sym=sym,qsuffix='USDT@binance' if sym in ['ETH','BTC'] else 'USDT@binancealt',date=[date(2021,10,15),date(2021,10,31)],ytrain_thresh=6,ytrain_periods=100,ytest_thresh=3,ytest_periods=100,vdelay=2,ytypes=["ys"],xtypes=["v","q99","q5","l"],qlags=[100,1000],request=['vbt'],models=["oneiflarger","rf","kmeans","mlp","dt"],randcv_n_iter=30,sl=2,tp=6,ts=None,vbtfreq="10Min") 
            res.extend(resml)
t1 = time.time()
print(t1-t0)
Audio(sound_file, autoplay=True)

In [ ]:
res